In [2]:
import numpy as np
import pandas as pd 
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import sys
import lmdb
import h5py
caffe_root = '/work/sagarj/caffe-rc5/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')
from collections import defaultdict
import caffe


In [4]:
model_root = "/work/sagarj/Work/BellLabs/caffe_models/caffe_model_1/"
# net_weights='caffe_beauty_augmented/caffe_model_beauty_augmented_iter_10000.caffemodel'
#net_weights='caffe_model_beauty_4_votes/caffe_model_1_iter_24732.caffemodel'
net_weights='smartAugment/caffe_model_beauty_city_smartAugment_iter_15000.caffemodel'
net_definition='caffenet_deploy_1.prototxt'

test_lmdb_path = '../Data/validation_lmdb_beauty_augmented' # Test LMDB database path
mean_file_binaryproto = '../Data/cityAugmentedMean.binaryproto' # Mean image file


In [5]:
caffe.set_mode_gpu()
model_def = model_root+net_definition
model_weights = model_root + net_weights

In [ ]:
net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)


In [ ]:
np.asarray(net.blobs['fc8'].data)

In [ ]:
# Extract mean from the mean image file
mean_blobproto_new = caffe.proto.caffe_pb2.BlobProto()
f = open(mean_file_binaryproto, 'rb')
mean_blobproto_new.ParseFromString(f.read())
mean_image = caffe.io.blobproto_to_array(mean_blobproto_new)
f.close()

In [ ]:
count = 0
correct = 0
labels = [] # (real,pred) -> int
preds = []
# labels_set = set()

lmdb_env = lmdb.open(test_lmdb_path)
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()

for key, value in lmdb_cursor:
    datum = caffe.proto.caffe_pb2.Datum()
    datum.ParseFromString(value)
    label = datum.label
    labels.append(label)
    image = caffe.io.datum_to_array(datum)
    image = image.astype(np.uint8)
    out = net.forward_all(data=np.asarray([image]) - mean_image).copy()

    plabel = out['prob']
    preds.append(plabel)
    count += 1

In [ ]:
preds[3]

In [ ]:
pred_label = [np.argmax(i) for i in preds ]

In [ ]:
pred_label[2] , labels[2]

In [ ]:
error_RF = mean_squared_error(labels, pred_label)

accuracy = np.mean(labels == pred_label)

precision , recall , fscore , _ = precision_recall_fscore_support(labels, pred_label)


print ( precision , recall , fscore)

In [ ]:
accuracy